# Tutoriaali: Karkeistetun molekyylidynamiikkasimulaation ajo GROMACS:lla

**Molekyylidynamiikka (MD)** on monipuolinen laskennallinen menetelmä, jolla voidaan simuloida makromolekyylien (esim. proteiinien) liikettä klassisesti.

Tässä harjoituksessa demonstroidaan miten lipidien, eli rasva-aineiden, dynamiikkaa ja itsejärjestymistä voidaan tutkia [GROMACS](https://www.gromacs.org/)-ohjelmiston avulla. Simulaation kiihdyttämiseksi hyödynnetään karkeistettuun [Martini](http://cgmartini.nl/)-voimakenttään perustuvaa mallia, jossa useampi vierekkäinen atomi mallinnetaan yhtenä hiukkasena, siis eräänlaisena "superatomina". Näin ollen simuloitavien hiukkasten määrä pienenee ja laskut nopeutuvat. Lisäksi simulaatiossa hyödynnetään rinnakkaislaskentaa *suurteholaskennan* (high-performance computing, HPC) periaatteen havainnollistamiseksi.

**Näin suoritat tämän tutoriaalin:**

* Valitse solu (alkaa `In [ ]:`) ja suorita käsky tai koodinpätkä painamalla samanaikaisesti `Shift+Enter`.
    * Tätä ennen muista kuitenkin lukea solua edeltävä teksti, sillä siinä kerrotaan mitä koodi/käsky tekee!
    * **Huom! Kohdassa 2. sinun tulee ensin muuttaa käskyn parametreja aikaisempien tulosteiden perusteella!**
* Odota, että komento suoriutuu (`In [*]:` tilalle ilmestyy esim. `In [11]:`)
* Siirry eteenpäin seuraavaan kohtaan (lue tekstit ja suorita komennot).

## 0. Valmistelut

Ensiksi meidän täytyy tehdä muutamia valintoja liittyen simulaatioomme:

1. *Minkälaisia lipidejä ja kuinka montaa molekyyliä haluamme tutkia?*
2. *Mitä liuotinta haluamme käyttää?*
3. *Kuinka suurta simulaatiokoppia haluamme käyttää?*

**Klikkaa allaolevaa solua ja paina `Shift+Enter` suorittaaksesi konfiguraatioskriptin `config.py`. Tämä avaa valikon, johon voit syöttää valintasi.** Ehdotetut arvot ovat hyvä lähtökohta, jos teet harjoituksen ensimmäistä kertaa. 

Mahdollisia liuottimia (solvent) ovat W = vesi, EOL = etanoli ja OD = oktadekaani. Lipidimolekyylien nimet ovat pitkiä ja monimutkaisia, joten lyhenteiden käyttö on yleistä. Simulaatiokopin koko (box size) vastaa kuutiollisen laatikon sivun pituutta yksikössä nanometri.

In [23]:
%run config.py

Dropdown(description='Lipid', index=4, options=('CPC', 'DLPC', 'DOPC', 'DPGS', 'DPPC', 'DPPG', 'DPSM', 'DTPC',…

Dropdown(description='Solvent', options=('W', 'EOL', 'OD'), value='W')

IntSlider(value=50, description='# of lipids', min=10)

FloatSlider(value=6.0, description='Box size', max=10.0, min=5.0, step=0.5)

Syöttämäsi arvot tallentuvat automaattisesti muuttujiin `lipid.value`, `solvent.value`, `nlip.value` ja `box.value`.

## 1. Solvatoidun lipidisysteemin rakentaminen

Kun simulaatiota koskevat valinnat on tehty rakennetaan solvatoitu lipidisysteemi. Lisätään valitsemasi määrä tiettyä lipidiä kuutiolliseen simulaatiokoppiin, jonka sivun pituus määriteltiin yllä. Tämä tapahtuu GROMACS-komennon `insert-molecules` avulla ja saatu tulos tallennetaan rakennetiedostoon `lipids.gro`. **Klikkaa alla olevaa solua ja suorita komento (`Shift+Enter`):**

In [ ]:
!gmx_mpi insert-molecules -o lipids.gro -ci ff22/{lipid.value}.gro -nmol {nlip.value} -box {box.value} -try 200

GROMACS tulostaa varsin paljon tietoa liittyen komennon suoritukseen. Oleellisin tieto on yleensä tulosteen lopussa. **Tärkeintä on aina tarkistaa, ettei tulosteen lopussa ole mainintaa virheistä (error).**

Seuraavaksi lipidilaatikkoomme tulee lisätä valittu liuotin. Alla oleva `solvate`-komento lisää koppiin niin paljon liuotinta kuin sinne mahtuu ottaen huomioon jo laatikkoon lisätyt lipidimolekyylit. Solvatoitu systeemi tallennetaan tiedostoon `solvated-lipids.gro`. Suorita alla oleva komento:

In [ ]:
!gmx_mpi solvate -cp lipids.gro -cs ff22/{solvent.value}.gro -o solvated-lipids.gro

**Hienoa, nyt sinulla on laatikko täynnä lipidimolekyylejä ja valittua liuotinta!**

## 2. Topologia

Ennen varsinaisen MD-simulaation aloittamista tulee meidän luoda nk. "topologia". Yksinkertaistaen, tämä on tiedosto, joka kertoo minkälaisia molekyylejä systeemissä on, kuinka monta, sekä mitä voimakenttää käytämme näiden vuorovaikutusten kuvaamiseksi.

Tässä harjoituksessa annetaan valmis mallitopologiatiedosto, mutta **ennen kuin voimme käyttää sitä tulee meidän muokata tiedostoa molekyylien lukumäärän osalta**.

**Etsi yllä olevista GROMACS-tulosteista kuinka monta lipidiä (Added X molecules) ja liuotinmolekyyliä (Number of solvent molecules: Y) mahtui oikeasti simulaatiokoppiin ja korvaa tekstit `LIPIDIMOLEKYYLIEN_LUKUMÄÄRÄ` ja `LIUOTINMOLEKYYLIEN_LUKUMÄÄRÄ` alla olevassa komennossa näillä arvoilla (X ja Y).**

Kun olet korvannut arvot, suorita alla oleva komento topologiatiedoston muokkaamiseksi.

**Huom! Jos lipidien ja liuotinmolekyylien lukumäärä topologiassa ei täsmää rakenteen kanssa, tulevat seuraavat askeleet epäonnistumaan, joten etsi ja korvaa arvot huolellisesti.**

In [ ]:
!bash init_files.sh {lipid.value} LIPIDIMOLEKYYLIEN_LUKUMÄÄRÄ {solvent.value} LIUOTINMOLEKYYLIEN_LUKUMÄÄRÄ

## 3. Energian minimointi

Ennen MD-simulaatiota on hyvä varmistaa, että systeemin lähtögeometria on järkevä. Atomit eivät saa olla liian lähellä toisiaan tai sidokset epäsuotuisasti vääntyneitä. Tämä varmistetaan minimoimalla systeemin energia. Käytännössä jokaiseen atomiin vaikuttava voima lasketaan voimakentän avulla ja atomeita liikutetaan voimien suuntaisesti kunnes ne saavuttavat minimin.

Energian minimointia varten luo syötetiedosto `em.tpr`, johon kootaan simulaatiparametrit (`em.mdp`), topologia (`topol.top`) ja systeemin rakenne (`solvated-lipids.gro`). **Tämä tapahtuu automaattisesti, kun suoritat alla olevan `grompp`-komennon:**

In [ ]:
!gmx_mpi grompp -f em.mdp -c solvated-lipids.gro -p topol.top -o em.tpr -maxwarn 10

Nyt meillä on syötetiedosto, joten voimme aloittaa energian minimoinnin! **Aja allaoleva `mdrun`-käsky:**

In [ ]:
!gmx_mpi mdrun -s em.tpr -v

Voimme havainnollistaa potentiaalienergian pienenemisen. **Aja komento alla, jotta saat tallennettua energian muuttumisen `ener.edr` output-tiedostosta tiedostoon nimeltä `energy.xvg`:**

In [ ]:
!echo "5" | gmx_mpi energy -f ener.edr -o energy.xvg

**Piirrä sitten data `matplotlib`-työkalun avulla ajamalla alla oleva Python-koodinpätkä:**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

energy = np.loadtxt('energy.xvg', comments=['#', '@'])
plt.plot(energy[50:, 0], energy[50:, 1])
plt.xlabel('Energiaminimointiaskel')
plt.ylabel('Potentiaalienergia (kJ/mol)')
plt.show()

Kuvaajasta näemme selkeästi miten systeemin potentiaalienergia pienenee ajon aikana. Nyt lähtögeometriamme on hieman stabiilimpi, joten olemme valmiit aloittamaan MD-simulaation syötetiedostojen valmistelun!

## 4. Molekyylidynamiikka

Ennen MD-simulaatiota tulee meidän jälleen valmistella syötetiedosto `md.tpr` kokoamalla yhteen energiaminimoitu rakenne (`confout.gro`), topologia (`topol.top`) ja MD-simulaation parametrit (`md.mdp`) `grompp`-käskyn avulla. **Suorita alla oleva komento:**

In [ ]:
!gmx_mpi grompp -f md.mdp -c confout.gro -p topol.top -o md.tpr -maxwarn 10

MD-simulaatiota nopeutetaan ajamalla työ rinnakkain usean laskentaytimen avulla. Komento `orterun -n $SLURM_CPUS_PER_TASK` käskee työtä käyttämään kaikkia ytimiä, jotka varattiin käyttöön, kun tämä Jupyter Notebook käynnistettiin (4 kpl, ellet muuttanut asetuksia).

Riippuen systeemisi koosta kestää simulaatio muutamasta kymmenestä sekunnista muutamaan minuuttiin. Voit ajaa simulaation useamman kerran ja vaihtaa `$SLURM_CPUS_PER_TASK`-muuttuja johonkin pienempään arvoon, esim 1 tai 2. Miten tämä vaikuttaa simulaation nopeuteen? Tulosteen lopussa löytyy tiedot kuinka monta nanosekuntia per päivä (ns/day) kyseisisillä parametreilla voi simuloida.

**Simulaation ajaminen hyödyntämällä useaa laskentaydintä (rinnakkaistaminen) on suurteholaskennan perusperiaate!**

In [ ]:
!orterun -n $SLURM_CPUS_PER_TASK --oversubscribe gmx_mpi mdrun -deffnm md -v

## 5. Visualisointi

**Hienoa, olet suorittanut molekyylidynamiikkasimulaation valitsemillesi lipidimolekyyleille jossakin liuottimessa!** Mielenkiintoisin kohta onkin nyt selvittää mitä simulaation aikana tapahtui.

Ennen visualisointia tulee meidän kuitenkin hieman korjata simulaation tuottamaa polkua `md.xtc` (systeemin aikakehitystä). Ajamasi simulaatio hyödynsi nk. *periodisia reunaehtoja* (periodic boundary conditions, PBC), mikä tarkoittaa, että molekyylit voivat ylittää simulaatiokopin reunat, jolloin ne päätyvät kopin vastakkaiselle puolelle. Tämä on kätevä tapa simuloida jatkuvia rakenteita kuten kiteitä ja liuoksia.

Systeemin visualisoiminen suoraan voi tästä syystä antaa harhaanjohtavan kuvan, että molekyylit vähitellen karkaavat toisistaan, johtaen pienenevään tiheyteen. Selkeän kuvan saamiseksi tulee meidän siis varmistaa, että reunat ylittävät molekyylit "käännetään" takaisin simulaatiokoppiin. **Tämä tapahtuu ajamalla seuraava `trjconv`-komento:**

In [ ]:
!echo "0" | gmx_mpi trjconv -f md.xtc -s md.tpr -o md_wrap.xtc -pbc mol

**Kokeile nyt visualisoida MD-simulaatio suorittamalla alla oleva koodinpätkä.**

In [ ]:
import MDAnalysis as mda
import nglview as nv

u = mda.Universe('md.gro', 'md_wrap.xtc')
view = nv.show_mdanalysis(u)
view.add_spacefill(lipid.value)
view.add_point(solvent.value)
view.add_unitcell()
view

Visualisaatiossa harmaat pallot kuvaavat lipidimolekyylien "häntiin" kuuluvia karkeistettuja atomeita, kun taas valkoiset ja värikkäät pallot kuvaavat pääryhmiin kuuluvia hiukkasia. Pienet pisteet ovat liuotinmolekyylejä. Voit pyöritellä ja suurentaa/pienentää rakennetta hiiren avulla ja kelata simulaatiopolkua ohjauspaneelista.

Kuten näet, pysyvät useimmat atomit simulaatiokopin sisällä koko simulaation aikana. Tekemämme korjaus saattaa kuitenkin aiheuttaa sen, että lipidien muodostama rakenne katkeaa reunojen yli, hankaloittaen rakenteen hahmottamista. **Kokeile ajaa alla oleva komento, joka siirtää lipidirakenteen kopin keskelle.**

In [ ]:
!echo "2 2 0" | gmx_mpi trjconv -f md.xtc -s md.tpr -o md_whole.xtc -pbc cluster -center

**Visualisoi polku sitten uudestaan (tällä kertaa ilman liuotinmolekyylejä):**

In [ ]:
import MDAnalysis as mda
import nglview as nv

u = mda.Universe('md.gro', 'md_whole.xtc')
view = nv.show_mdanalysis(u)
view.add_spacefill(lipid.value)
view.add_unitcell()
view

## 6. Termodynaaminen ja geometrinen tulkinta

**Eli mitä tässä oikein tapahtui?**

Simulaation alussa lipidit ovat siellä täällä, mutta ajan myötä ne näyttävät järjestyvän itsestään! Ilmiölle on termodynaaminen selitys. Lipidit ovat nk. *amfifiilisiä* molekyylejä, eli niillä on **hydrofiilinen (poolinen) pääryhmä** sekä **hydrofobinen (pooliton) häntä**. Riippuen liuottiminen poolisuudesta lipidit haluavat siis järjestyä siten, että saman poolisuuden omaavat ryhmät ovat lähellä toisiaan.

Lipidien itsejärjestyminen on spontaani prosessi sillä se **minimoi systeemin vapaaenergian $\Delta G$**. Yleensä termodynaamiset systeemit pyrkivät kasvattamaan *entropiaan* ("epäjärjestystä"), mutta tässä tapauksessa systeemin järjestys kasvaa, eli entropia pienenee! Vapaaenergian muutokseen, mikä määrää tapahtuuko jokin prosessi itsestään vai ei, vaikuttaa kuitenkin olennaisesti myös entalpia. Entalpian muutos onkin tässä tapauksessa varsin negatiivinen sillä lipidien järjestyminen johtaa suotuisampaan elektrostaattiseen vuorovaikutukseen ryhmien välillä. Näin ollen vapaaenergian muutos on kokonaisuudessaan negatiivinen:

$$\Delta G=\Delta H-T\Delta S<0$$

Poolisuuden määräämän järestyksen lisäksi **lipidien geometria** vaikuttaa siihen minkälaisia rakenteita muodostuu. *Kriittinen pakkautumisparametri* (CPP) on tyypillisesti käytetty mitta, jolla voidaan ennustaa laadullisesti itsejärjestyneen lipidirakenteen muoto,

$$\mathrm{CPP}=\frac{V_t}{A_h\cdot l_t}$$

missä $V_t$ ja $l_t$ ovat hännän tilavuus ja pituus, kun taas $A_h$ on pääryhmän poikkileikkauksen pinta-ala. Poolisen liuottimen tapauksessa lipidit joilla on iso pääryhmä ($\mathrm{CPP}\leq1/3$) muodostavat tyypillisesti pallomaisia *misellejä*, kun taas lieriömäiset rakenteet ja *vesikkelit*, eli pallomaiset kaksoiskalvot, ovat mahdollisia, jos pääryhmä on hieman pienempi ($1/3<\mathrm{CPP}\leq1/2$, esim. LPC). Jos lipidi on muodoltaan lieriö (pään poikkileikkaus on n. yhtä suuri kuin hännän, $1/2<\mathrm{CPP}\leq1$, esim. DPPC), muodostuu yleensä erinäisiä kaksoiskalvoja.

Toisaalta, poolittomissa liuottimissa lipidit voivat muodostaa *käänteismisellejä*, joissa hydrofoobiset hännät osoittavat kohti liuotinta. Inversio on mahdollista myös poolisissa liuottimissa, jos lipidien pääryhmä on hyvin pieni ($\mathrm{CPP}>1$, esim. POPE). Tällöin itsejärjestyneissä rakenteissa liuotin on käytännössä vangittuna hydrofiilisten pääryhmien muodostamiin aukkoihin.

## 7. Kokeile simulaatiota uudestaan!

Vaihda valitsemaasi lipidiä ja/tai luotinta ja tutki saatko toistettua joitain yllä mainituista itsejärjestyneistä rakenteista. Huomaa, että myös lipidien konsentraatio vaikuttaa tuloksiin – isojen rakenteiden kuten kaksoiskalvojen muodostuminen on epätodennäköistä, jos lipidimolekyylien lukumäärä on liian pieni suhteessa simulaatiokopin kokoon.

**Aloittaaksesi puhtaalta pöydältä voit ajaa alla olevan skriptin poistaaksesi kaikki tähän asti luodut tiedostot. Voit myös tyhjentää kaikki tämän sivun tulosteet yläpalkista `Cell > All Output > Clear`.**

In [ ]:
!bash clean.sh